In [1]:
import pythran
%load_ext pythran.magic

In [2]:
def get_complex_random_array(dimensions):
    real_part = np.random.rand(*dimensions)
    imaginary_part = np.random.rand(*dimensions)
    complex_array = real_part + imaginary_part * 1.0j
    return complex_array

In [3]:
%%pythran -fopenmp -Ofast
#pythran export pythran_exp_1j(complex64[:, :, :])

def pythran_exp_1j(prototype):
    import numpy as np
    shape = prototype.shape
    result = np.zeros((shape[0], shape[1], shape[2]), dtype=np.complex64)
    "omp parallel for"
    for i in range(shape[0]):
        for j in range(shape[1]):
            for k in range(shape[2]):
                result[i, j, k] = (np.cos(prototype[i, j, k].real) + 1.0j*np.sin(prototype[i, j, k].real)) * \
                                                            np.exp(-prototype[i, j, k].imag)

    return result

In [4]:
import numpy as np
from numba import complex64, prange, jit, types

@jit(['complex64[:, :, :](complex64[:, :, :])'],
     nopython=True, cache=True, nogil=True, fastmath=True, parallel=True)
def numba_exp_1j(prototype):
    shape = prototype.shape
    result = np.zeros((shape[0], shape[1], shape[2]), dtype=np.complex64)
    for i in prange(shape[0]):
        for j in range(shape[1]):
            for k in range(shape[2]):
                result[i, j, k] = (np.cos(prototype[i, j, k].real) + 1.0j*np.sin(prototype[i, j, k].real)) * \
                                                            np.exp(-prototype[i, j, k].imag)

    return result

In [5]:
def numpy_exp_1j(prototype):
    return np.exp(1j * prototype)

In [7]:
!python3 -m numpy.f2py --f90flags='-Ofast -lgomp -fopenmp' -lgomp -c expj.f90 -m expj

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "expj" sources
f2py options: []
f2py:> /tmp/tmpsh9mosen/src.linux-x86_64-3.8/expjmodule.c
creating /tmp/tmpsh9mosen/src.linux-x86_64-3.8
Reading fortran codes...
	Reading file 'expj.f90' (format:free)
Post-processing...
	Block: expj
			Block: expj_external_module
In: :expj:expj.f90:expj_external_module
get_useparameters: no module omp_lib info used by expj_external_module
Post-processing (stage 2)...
Building modules...
	Building module "expj"...
		Creating wrapper for Fortran subroutine "expj_external_module"("expj_external_module")...
		Constructing wrapper function "expj_external_module"...
		  expj_external_module(source,result)
	Wrote C/API module "expj" to file "/tmp/tmpsh9mosen/src.linux-x86_64-3.8/expjmodule.c"
	Fort

In [13]:
import datetime
import numpy as np
import expj

def test_exponents():
    numpy_time = []
    pythran_time = []
    numba_time = []
    fortran_time = []
    for i in range(100):
        prototype = get_complex_random_array((50, 300, 710)).astype(np.complex64)
        fortran_result = np.zeros((50, 300, 710)).astype(np.complex64)

        #numpy test zone
        numpy_start = datetime.datetime.now()
        res = numpy_exp_1j(prototype)
        numpy_end = datetime.datetime.now()
        numpy_time.append(numpy_end - numpy_start)

        #pythran test zone
        pythran_start = datetime.datetime.now()
        res = pythran_exp_1j(prototype)
        pythran_end = datetime.datetime.now()
        pythran_time.append(pythran_end - pythran_start)
        
        #numba test zone
        numba_start = datetime.datetime.now()
        res = numba_exp_1j(prototype)
        numba_end = datetime.datetime.now()
        numba_time.append(numba_end - numba_start)
        
        #fortran test zone
        fortran_start = datetime.datetime.now()
        expj.expj_external_module(prototype, fortran_result)
        fortran_end = datetime.datetime.now()
        fortran_time.append(fortran_end - fortran_start)
        

    print('numpy time: ', np.sum(numpy_time))
    print('pythran time: ', np.sum(pythran_time))
    print('numba time: ', np.sum(numba_time))
    print('fortran time: ', np.sum(fortran_time))
    
    print('pythran speed up = ', np.sum(numpy_time) / np.sum(pythran_time))
    print('numba speed up = ', np.sum(numpy_time) / np.sum(numba_time))
    print('fortran speed up = ', np.sum(numpy_time) / np.sum(fortran_time))
    
test_exponents()

numpy time:  0:00:22.749066
pythran time:  0:00:04.340731
numba time:  0:00:41.621528
fortran time:  0:00:19.101882
pythran speed up =  5.240837545565482
numba speed up =  0.5465696982580746
fortran speed up =  1.1909332284640854


In [3]:
!mpifort -Ofast -fopenmp expj_internal.f90
!./a.out

   3.62774706    


In expj_internal.f90 we have the same parameters ((50, 300, 710) array for 100 times) and have the same value to output (counting time), so the result from above can be compared with others.